In [28]:
import toml, time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from typing import Dict, Optional
from binance.client import Client
from datetime import datetime, timezone

plt.rcParams["figure.figsize"] = (20,10)
plt.rcParams.update({'font.size': 18})

In [29]:
cfg = toml.load("configuration.toml")
pkey = cfg["auth"]["pkey"]
skey = cfg["auth"]["skey"]

client = Client(pkey, skey)

In [ ]:
# Number of trades to get
N = 1_000_000

# Get initial set of trades, starting from now
historical_trades = client.get_historical_trades(symbol="DOGEUSDT")

# min_id from the trades
start_id = historical_trades[0]["id"]

# Putting everything into a pandas dataframe
df = pd.DataFrame.from_records(historical_trades)

n = 500

# So we dont hit the rate limit of 1200 / min
min_period = .05
norm_period = .5

t0 = time.perf_counter()
t = t0
try:
    while n < N:
        curr_id = start_id - n

        trades_chunk = client.get_historical_trades(symbol="DOGEUSDT", fromId=str(curr_id))
        df = pd.concat([df, pd.DataFrame.from_records(trades_chunk)], ignore_index=True)

        n += 500

        # if we looped faster than the period for our rate limiter (.05 s minimum), wait until
        # we slowed down enough like GLB
        t += norm_period
        time.sleep(max(0, t - time.perf_counter()))
        print(curr_id % 10000)
except Exception as e:
    print(e)
    print(f"execution time: {time.perf_counter() - t0}")

In [ ]:
# Sort the DF by id (ie chronological) and reset the index so pandas index ordering fits the data
df.sort_values(by=["id"], inplace=True)
df.reset_index(drop=True, inplace=True)

In [ ]:
df.to_pickle("./previous_1e6_trades.pkl")

In [ ]:
trade_datetimes = sorted([datetime.fromtimestamp(timestamp / 1000, timezone.utc) for timestamp in df["time"]])
trade_diffs = np.diff(sorted(df["time"]))